# 母集団と標本

# R演習

## 平均・分散・標準偏差

### 利用するデータ1

引き続き`palmerpenguins`のデータを利用する

In [ ]:
library(pacman)
p_load(palmerpenguins)
str(penguins)

### 算術平均

算術平均を算出する関数として`mean()`関数がある.
`penguins`のデータには欠測値があるので, `na.rm = TRUE`を引数に追加する.

データフレームを直接引数として取ることはできない.
小数点以下の値を丸めるには`round()`関数を利用する.
第2引数で小数点以下何桁表示するかを与える.

In [ ]:
mean(penguins$bill_length_mm, na.rm = TRUE)

### `summary()`関数を用いて平均を求める

`summary()`関数でも算術平均を知ることができる.
`summary()`関数はベクトルでもデータフレームでも良い.

In [ ]:
summary(penguins$bill_length_mm)

### ヒストグラムに算術平均を加える

`ggplot2`で縦線を加えるにはグラフに`geom_vline()`を追加する.
`xintercept`でx軸のどこに線を引くのかを与える. ここでは,
算術平均を計算し, 与えている.
線は`linetype`で指定できるため点線を指定し,
`colour`で線の色を赤に指定した.

In [ ]:
p_load(ggplot2)
ggplot(penguins) +
  aes(x = bill_length_mm) +
  geom_histogram() +
  geom_vline(
    xintercept = 
      mean(penguins$bill_length_mm,
           na.rm = TRUE
      ),
    linetype = "dashed",
    colour   = "red"
  )

### 算術平均からの偏差の和

算術平均からの偏差の和が0になることを検証する. 数値計算上,
割り切れない場合はごく小さい値が算出されることもあることに注意.

In [ ]:
sum(
  penguins$bill_length_mm -
    mean(penguins$bill_length_mm,
         na.rm = TRUE
    ),
  na.rm = TRUE
)

### 標本分散

標本分散を求めるために各データから平均を引いて2乗し, その平均を取る.
Rにデフォルトで搭載されている`var()`関数は標本分散ではない(以降扱う不偏分散($n$ではなく,
$n-1$で除す)).

In [ ]:
mean(
  (penguins$bill_length_mm -
    mean(penguins$bill_length_mm,
      na.rm = TRUE
    )
  )^2,
  na.rm = TRUE
)

### 不偏分散

不偏分散を求めるには`var()`関数を用いる.

In [ ]:
var(penguins$bill_length_mm,
    na.rm = TRUE)

### 標本分散を見やすくコーディングする(応用)

パイプ演算子を利用するとコードが読みやすくなる.

In [ ]:
p_load(magrittr)
penguins %$%
  subtract(
    bill_length_mm,
    bill_length_mm |>
      mean(na.rm = TRUE)
  ) |>
  raise_to_power(2) |>
  mean(na.rm = TRUE)

### 標本分散の関数化

以降利用しやすいように標本分散を関数にする.

In [ ]:
sample_var <- function(x) {
  subtract(
    x,
    x |>
      mean(na.rm = TRUE)
  ) |>
    raise_to_power(2) |>
    mean(na.rm = TRUE)
}
sample_var(penguins$bill_length_mm)

### 標本標準偏差

標本標準偏差を求めるために各データから平均を引いた値の平方平均(標本分散)の平方根を求める.
Rにデフォルトで搭載されている`sd()`関数は標本標準偏差ではない(以降扱う不偏分散の平方根).

In [ ]:
sqrt(
  mean(
    (penguins$bill_length_mm - 
      mean(penguins$bill_length_mm,
           na.rm = TRUE
           )
      )^2,
      na.rm = TRUE
  )
)

### 不偏分散の平方根

不偏分散の平方根を求めるには`sd()`関数を利用する.
不偏分散の平方根は不偏性を持っていないことに注意.

In [ ]:
sd(penguins$bill_length_mm,
   na.rm = TRUE)

### 標本標準偏差を見やすくコーディングする(応用)

パイプ演算子を利用するとコードが読みやすくなる.

In [ ]:
penguins %$% 
  subtract(bill_length_mm,
           bill_length_mm |> 
             mean(na.rm = TRUE)) |> 
  raise_to_power(2) |> 
  mean(na.rm = TRUE) |> 
  sqrt()

### 標本標準偏差の関数化

以降利用しやすいように標本標準偏差を関数にする.

In [ ]:
sample_sd <- function(x){
  subtract(x,
           x |> 
             mean(na.rm = TRUE)) |> 
    raise_to_power(2) |>  
    mean(na.rm = TRUE) |> 
    sqrt()
}
sample_sd(penguins$bill_length_mm)

### データの標準化

データを標準化するには`scale()`関数を利用する. 出力結果は少し厄介.

In [ ]:
head(scale(penguins$bill_length_mm))

## 大数の弱法則と中心極限定理

### 利用するデータ2

今回はggplot2パッケージに含まれる`diamonds`というデータセットを利用する.
約54,000行10列からなるデータであり, ダイヤモンドの価格(\$)やカラット,
カットのクオリティなどの属性値が含まれる.

このデータを母集団に見立てて, サンプリングの学習を行う.
母集団がやや小さいことやサンプルサイズや標本数は計算の都合上やや小さいことに注意.
まずは`diamonds`データの列名を`names()`関数を利用して確認する.

In [ ]:
names(diamonds)

### データの先頭6行確認する

`head()`関数を利用して`diamonds`データの先頭6行を確認する.

In [ ]:
head(diamonds)

### データ構造を確認する

`dplyr`パッケージの`glimpse()`関数を利用して`diamonds`データのデータ構造を確認する.
行数や列数, 各列のデータ型などを把握する.

In [ ]:
p_load(dplyr)
dplyr::glimpse(diamonds)

### データの概要を把握する

`summary()`関数を利用してデータの概要を把握する.

In [ ]:
summary(diamonds)

### データ内の分散・標準偏差を求める

今回は`diamonds`データを母集団とみなすので, データの標本分散を求め,
母分散とみなす. 列は`price`を利用する.
後で利用するために母平均もオブジェクトに格納する.

In [ ]:
var_pop  <- sample_var(diamonds$price)
var_pop

### データのヒストグラムを描画する

`diamonds`データの`price`の列のヒストグラムを描画する.
価格や年収のデータに多い右に裾の長いデータになっている.

In [ ]:
ggplot(data = diamonds) +
  aes(x = price) +
  geom_histogram()

### データの無作為抽出

`dplyr::slice_sample()`関数を利用してデータの無作為抽出を行う.
`diamonds`データからサイズ100のサンプルを非復元抽出(`replace = FALSE`)し,
`ss`というオブジェクトに格納する.

無作為抽出の際に疑似乱数を利用しているので,
乱数の種を指定して結果を再現できるようにする.
`withr`パッケージの`with_seed()`関数を利用する.
第1引数で任意の乱数の種を指定し, 第2引数に実行する関数.

In [ ]:
p_load(withr)
ss <- withr::with_seed(
  2718,
  dplyr::slice_sample(
    diamonds, 
    n       = 100, 
    replace = FALSE
    )
  )

### 標本の概要を確認

`summary()`関数を利用して標本の概要を把握する.

In [ ]:
summary(ss)

### 標本の平均, 分散を母集団のそれと比較する

標本平均, 標本分散をそれぞれ母平均, 母分散と比較する.

In [ ]:
mean(ss$price)

### 標本データのヒストグラム

標本データもヒストグラムを描画する. 形状は似ているが,
母集団を再現するには至らない.

In [ ]:
ggplot(data = ss) +
  aes(x = price) +
  geom_histogram()

### 標本の数を増やす

サイズ500のサンプルを1000個(サンプル数1000)作成する.
繰り返し行う処理などは`tidyverse`パッケージに含まれている`purrr`パッケージを利用すると便利.
`purrr::map()`関数は, 第1引数で指定したリストに対して,
第2引数以降で書かれた関数を実行してくれる.
ここでは第2引数で`~`を使って無名関数を定義している. 結果は`List`形式.

In [ ]:
p_load(purrr)
ms <- with_seed(
  2718,
  purrr::map(
    1:1000,
    \(x) slice_sample(
      diamonds,
      n       = 500,
      replace = FALSE
      )
    )
  )

### 各標本の標本平均を求める

`dplyr::summarise()`関数はデータを要約する関数.
`sample_mean`という列名を作成し,
そこに`ms`オブジェクトの中の各`price`の値の平均を格納する.
最後に結果のリストを`list_rbind()`関数を利用して,
各標本の平均を1つのデータフレームに縦方向に整理する.

In [ ]:
ms_mean <- purrr::map(
  ms, 
  dplyr::summarise,
  sample_mean = mean(price)
  ) |> 
  list_rbind()

### 標本平均の平均を求める

標本平均の平均を求め, 母平均と比較する.
標本平均の標本平均値は母平均ととても近い値になる.
サンプルサイズをさらに増やすと標本平均は母平均と一致する.

In [ ]:
mean(ms_mean$sample_mean)

### 標本平均の分散

標本平均の分散は$\frac{\sigma^2}{n}$と近似する.

In [ ]:
sample_var(ms_mean$sample_mean)

### 標本平均のヒストグラム

標本平均の分布は平均$\mu$,
分散$\frac{\sigma^2}{n}$の正規分布で近似できる.
中心極限定理は元の分布を問わない.

In [ ]:
ggplot(data = ms_mean) +
  aes(x = sample_mean) +
  geom_histogram(
    aes(y = after_stat(density)),
    fill  = "blue",
    alpha = 0.2
    ) +
  geom_density(fill  = "blue") +
  geom_vline(xintercept = mean_pop) +
  stat_function(
    fun  = dnorm,
    args = list(
      mean = mean_pop,
      sd   = sqrt(var_pop / 500)),
    size = 2
    )

## 信頼区間

### 各サンプルの平均と95%信頼区間を計算する

`map()`関数と`list_rbind()`関数を利用して,
各サンプルの平均と95%信頼区間を計算し, データフレームへ変換する.
`.id = **`を指定することで,
`rerun()`関数で作成したリストの各リストの名前を`**`という名前の列に追加する.
ここでは各リストに通し番号が振られているので,
通し番号を`number`という列に格納している.

In [ ]:
ms_ci <- map(
  ms,
  summarise,
  lowerCI = mean(price) + 
    qnorm(0.025) * sqrt(var_pop / length(price)),
  mean    = mean(price),
  upperCI = mean(price) + 
    qnorm(0.975) * sqrt(var_pop / length(price))) |> 
  list_rbind(names_to = "number")

### 信頼区間とパラメーター

信頼区間にパラメーターが含まれるかどうかを1か0かで判定する.
データフレームで列を操作する(新しい列を作る,
列の中身を変更するetc…)場合には,
`dplyr`パッケージの`mutate()`関数を利用する.
第1引数に利用するデータフレーム,
第2引数に`** =`の`**`で列の名前を指定し, `=`の後で操作方法を指定する.
ここでは, さらに`dplyr`パッケージの`if_else()`関数を利用して,
信頼区間の下限より大きく, 上限より小さい場合に1,
そうでない場合に0を格納するように指示している.

In [ ]:
ci_TF <- dplyr::mutate(
  ms_ci, 
  TF = lowerCI <= mean_pop & upperCI >= mean_pop
  )

### 信頼区間にパラメーターが含まれる割合

先程作成した, TFの平均を計算する.
この平均は信頼区間にパラメーターが含まれる割合である.
実際に95%に近い値が算出されている.

In [ ]:
summarise(ci_TF, mean(TF))

### 

標本をすべて表示するのは多すぎるので, 先頭の100標本だけ表示する.
`geom_errobar()`関数は信頼区間などを示す際に利用できる.
`aes()`内で下限を`ymin`で, 上限を`ymax`で与える.

In [ ]:
ggplot(data = head(ci_TF, 100)) +
  aes(x      = number, 
      y      = mean,
      colour = TF) +
  geom_point() +
  geom_errorbar(
    aes(ymin = lowerCI,
        ymax = upperCI), 
    width = .2) +
  geom_hline(
    yintercept = mean_pop, 
    linetype   = "dashed", 
    colour     = "black")